Introduction
- Today's learning objective
- Next week
- CTA

 <img src="go.nannyml.com_be-a-design-partner.png"/>

# Context

Usecase idea:
https://api.openml.org/d/42729

Actual dataset source:
https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

Data dictionary:
https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf

# Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor, plot_importance

import nannyml as nml

# Getting the data

In [ ]:
# Read data from url
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2016-12.parquet"
columns = ['lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance', 'VendorID', 'payment_type', 'fare_amount', 'tip_amount']
data = pd.read_parquet(url, columns=columns) # ignore the "total amount"

# Preprocessing the data

In [ ]:
# Data selection
data = data.loc[data['payment_type'] == 1,].drop(columns='payment_type') # Credit cards

# Data cleaning
data = data.sort_values('lpep_pickup_datetime').reset_index(drop=True)
categoric_columns = ['PULocationID', 'DOLocationID', 'VendorID']
data[categoric_columns] = data[categoric_columns].astype('category')

# Feature engineering
data['pickup_time'] = data['lpep_pickup_datetime'].dt.hour

In [ ]:
# Create data partition
data['partition'] = pd.cut(
    data['lpep_pickup_datetime'],
    bins= [pd.to_datetime('2016-12-01'),
           pd.to_datetime('2016-12-08'),
           pd.to_datetime('2016-12-16'),
           pd.to_datetime('2017-01-01')],
    right=False,
    labels= ['train', 'test', 'prod']
)

# Set target and features & date column
target = 'tip_amount'
features = [col for col in data.columns if col not in [target, 'lpep_pickup_datetime', 'partition']]

# Split the data
X_train = data.loc[data['partition'] == 'train', features]
y_train = data.loc[data['partition'] == 'train', target]

X_test = data.loc[data['partition'] == 'test', features]
y_test = data.loc[data['partition'] == 'test', target]

X_prod = data.loc[data['partition'] == 'prod', features]
y_prod = data.loc[data['partition'] == 'prod', target]

# Train a model

In [ ]:
# Fit the model
model = LGBMRegressor(random_state=111)
model.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Evaluate the model

In [ ]:
# Make baseline predictions
y_pred_train_baseline = np.ones_like(y_train) * y_train.mean()
y_pred_test_baseline = np.ones_like(y_test) * y_train.mean()

# Measure train, test and baseline performance
mae_train = mean_absolute_error(y_train, y_pred_train).round(4)
mae_test = mean_absolute_error(y_test, y_pred_test).round(4)

mae_train_baseline = mean_absolute_error(y_train, y_pred_train_baseline).round(4)
mae_test_baseline = mean_absolute_error(y_test, y_pred_test_baseline).round(4)

In [ ]:
# Create performance report 
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,4))

title1 = 'Train MAE: {} (<> {})'.format(mae_train, mae_train_baseline) 
ax1.set(title=title1, xlabel='y_train', ylabel='y_pred')
ax1.plot(y_train, y_train, color='red', linestyle=':')
ax1.scatter(y_train, y_pred_train, alpha=0.1)

title2 = 'Test MAE: {} (<> {})'.format(mae_test, mae_test_baseline)
ax2.set(title=title2, xlabel='y_train', ylabel='y_pred')
ax2.plot(y_test, y_test, color='red', linestyle=':')
ax2.scatter(y_test, y_pred_test, alpha=0.1)

plt.show()

fig, ax = plt.subplots()
plot_importance(model, ax=ax)
plt.show()

# Deploy the model

In [ ]:
y_pred_prod = model.predict(X_prod)

---

# Analysing ML model performance in production

## 1. Estimating model performance
Articles:
- [DLE - direct loss estimation for regression](https://towardsdatascience.com/you-cant-predict-the-errors-of-your-model-or-can-you-1a2e4a1f38a0)

- [CBPE - confidence-based performance estimation for classification](https://towardsai.net/p/l/estimating-model-performance-without-ground-truth)

## 2. Multivariate data drift

Articles:
- [PCA based Data Reconstruction](https://towardsdatascience.com/detecting-covariate-shift-a-guide-to-the-multivariate-approach-c099bd1891b9)

## 3. Univariate data drift
Article:
- [Univariate Drift Detection Methods](https://nannyml.readthedocs.io/en/stable/how_it_works/univariate_drift_detection.html)

# Afterthoughts

## False alarms

## Realised performance

## Prediction drift

## Target drift